In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import pyrealsense2 as rs
import serial
import numpy as np
import time
from threading import Thread
import threading
import DataDeal265 as dd
import struct
import queue
import cv2
import numpy as np
import ipywidgets
import math
from Fire import ShapeDetector
CopterTakingOff =1
TargetPosition = [0.0,0.0,0.0]

#图像大小
cap_x=360
cap_y=240

# X坐标 Y坐标
# 路径
cp1=[0.0,0.0]#
cp2=[-1.5,0.0]#
cp3=[-1.5,1.4]#

cp4=[-2.5,1.4]#
cp5=[-1.5,1.4]
cp6=[-1.5,2.4]

cp7=[-0.5,2.4]#
cp8=[-0.5,3.4]
cp9=[-2.5,3.4]#

cp10=[-1.5,3.4]
cp11=[-1.5,0.0]
cp12=[-0.0,0.0]
#三个区域坐标
A=cp4
B=cp7
C=cp9
# 路径点入队
roqu=queue.Queue()
roqu.put(cp1)
roqu.put(cp2)
roqu.put(cp3)#
roqu.put(cp4)
roqu.put(cp5)#
roqu.put(cp6)
roqu.put(cp7)#
roqu.put(cp8)
roqu.put(cp9)
roqu.put(cp10)
roqu.put(cp11)
roqu.put(cp12)
# 图像检测器
sd = ShapeDetector()

font_scale=1.5 #字体大小
font=cv2.FONT_HERSHEY_PLAIN#字体类型
imageFlag=True
CE=0.1#坐标误差判断
KNOWN_DISTANCE = 10 #飞行高度分米

In [2]:

# 字符串对比
def StrComparison(str1,str2):
    n = len(str1)
    res = []
    for x in str1:
        if x in str2:
            res.append(x)
    return (n-len(res))

def fireIdentification(cap,wid):
    triangle = 0
    square = 0
    circle = 0
    while True:
        ret,image = cap.read()    
        # 颜色空间转换
        hsv=cv2.cvtColor(image,cv2.COLOR_BGR2HSV)

        #设置阈值
        lower_red = np.array([0, 43, 46])
        upper_red = np.array([10, 255, 255])
        # 根据侷值构建掩模
        mask0=cv2.inRange(hsv,lower_red,upper_red)
        #设置阈值
        lower_red = np.array([156, 43, 46])
        upper_red = np.array([180, 255, 255])
        # 根据侷值构建掩模
        mask1 = cv2.inRange(hsv,lower_red,upper_red)
        # 合并掩模
        mask = mask0 + mask1

        # 稍微模糊，
        blurred = cv2.GaussianBlur(mask, (5, 5), 0)

        # 在阈值图像中查找轮廓并初始化
        # 形状检测器
        cnts,_ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

        # 找到最大轮廓
        max_idx = sd.maxContour(cnts)
        if max_idx: 
            c=cnts[max_idx]
            M = cv2.moments(c)

            # 计算轮廓的中心，然后检测
            #仅使用轮廓的形状
            cX = int((M["m10"] / M["m00"]))
            cY = int((M["m01"] / M["m00"]))
            shape = sd.detect(c)
            
            # 多次判断结果
            if shape == "triangle":
                triangle+=1
                if triangle>10:
                    print("\n"+"识别形状"+shape+"按照比赛要求火情最小")
                    break
            if shape == "square":
                square+=1
                if square>10:
                    print("\n"+"识别形状"+shape+"按照比赛要求火情最大")
                    break
            if shape == "circle":
                circle+=1
                if circle>10:
                    print("\n"+"识别形状"+shape+"按照比赛要求火情次之")
                    break

            # 在图像上绘制轮廓和形状名称
            cv2.drawContours(image, [c], -1, (0, 255, 0), 2)
            cv2.putText(image, shape, (cX, cY), cv2.FONT_HERSHEY_SIMPLEX,
                0.5, (255, 255, 255), 2)

        # 显示输出图像
        sd.showImage(mask,image,wid)


#路径点更新图像识别线程
def Router(cap,wid):
    global routeNodeIndex
    global SendTargetPos
    global CopterLanding
    global LaserArray
    global LaserDistance
    global FlightMode
    global CopterTakingOff
    global routeStartFlag
    global targetx  
    global targety
    while True:
        SendTargetPos = 1       
        if math.sqrt((targetx - pos_Y)**2+(targety + pos_X)**2)<CE and routeStartFlag:
            if imageFlag:
                if math.sqrt((A[0] - pos_Y)**2+(A[1] + pos_X)**2)<CE or math.sqrt((B[0] - pos_Y)**2+(B[1] + pos_X)**2)<CE or math.sqrt((C[0] - pos_Y)**2+(C[1] + pos_X)**2)<CE:
                    fireIdentification(cap,wid)
                   
            if not roqu.empty():
                TargetPosition=roqu.get()
                routeNodeIndex+=1
                targetx = float(TargetPosition[0])
                targety = float(TargetPosition[1])
                print ("route node %d: x : %.1f , y : %.1f  " %(routeNodeIndex, targetx, targety))
                time.sleep(1)
            else:
                CopterTakingOff = 1
                CopterLanding = 1
                routeNodeIndex= 1
                print ("Landing") 
                break
    

           

#串口通信线程
def PortCom(port):
    global pipe
    global cfg
    global SendTargetPos
    global CopterLanding
    global CopterTakingOff
    global _265Ready
    global GetOnceCmd
    global routeNodeIndex
    global routeStartFlag

    while(True):
#         size=port.inWaiting()
#         if(size!=0):

        response = port.readline()
        if(response !=None):
            port.flushInput()        
            CmdStr1 = str(b'Start265\n')
            CmdStr2 = str(b'Departures\n')
            CmdStr3 = str(b'Refresh265\n')
            CMD = str(response)            
            #刷新265
            if( ( StrComparison( CMD ,CmdStr1 )<=1)  and  GetOnceCmd ==False):
                print(StrComparison( CMD ,CmdStr1 ),response,CMD)
                # Declare RealSense pipeline, encapsulating the actual device and sensors
                pipe = rs.pipeline()
#                 try:
#                     pipe.stop()
#                 except:
#                     print("Error1")
                # Build config object and request pose data
                cfg = rs.config()
                cfg.enable_stream(rs.stream.pose,rs.format.any,framerate=200)
                # 使用请求的配置开始流式传输
                pipe.start(cfg)
                dd.initData()
                SendTargetPos = 0
                CopterLanding = 0
                _265Ready=True
                GetOnceCmd =True 
                routeStartFlag = True

            elif( ( StrComparison(CMD ,CmdStr2 )<=1)  and  CopterTakingOff == 1 ):
                print(StrComparison(CMD ,CmdStr2 ),response,CMD)
                print("Get!")
                # 打开摄像头  
                cap,wid = sd.openCamera()
                router = Thread(target=Router, args=(cap,wid,))
                router.start()
                CopterTakingOff = 0
                
            elif( StrComparison(CMD ,CmdStr3 )<=1):
                _265Ready = False
                GetOnceCmd = False
                routeNodeIndex= 1
                CopterTakingOff =1
                routeStartFlag = False
                print("ReStart!")
                print( StrComparison(CMD ,CmdStr3 ),response,CMD)
                try:
                    pipe.stop()
                    time.sleep(1.0)               
                except:
                    print("Error2")  
            response = 0
            CMD = 0
       
                        
        time.sleep(0.02) 

if __name__ == '__main__':
    global routeNodeIndex # 路由节点索引
    global SendTargetPos  #发送目标位置
    global CopterLanding  # 降落 
    global LaserArray     # 雷达
    global LaserDistance  # 激光器距离
    global FlightMode     #飞行模式
    global pipe
    global _265Ready      # 265准备就绪
    global GetOnceCmd     # 获取一次命令
    global CheckSum       # 校验和
    global targetx        # 目标x
    global targety        # 目标y
    global dataBuf

    # 串口
    port = serial.Serial(port="/dev/ttyAMA0",baudrate=230400,stopbits=1,parity=serial.PARITY_NONE,timeout=1000)
    kernel = np.ones((4, 4), np.uint8) #膨胀算子
    thresValue=50                      #阈值
    
    #串口通信线程
    thread_Serial = Thread(target=PortCom, args=(port,))
    thread_Serial.start()
    
    #导入
    TargetPosition=roqu.get()
    targetx = TargetPosition[0]
    targety = TargetPosition[1]
    
    routeNodeIndex = 1
    _265Ready =False
    GetOnceCmd =False
    CheckSum =0 
    dataBuf = [0]*65
    LaserArray =0
    LaserDistance=0.0
    FlightMode =0
    try:
        while(True):
            if _265Ready:
                # 等待t265相机的下一组帧
                frames = pipe.wait_for_frames()
                # 获取姿势帧
                pose = frames.get_pose_frame()
                if pose:   
                    # 将一些姿势数据打印到终端
                    data = pose.get_pose_data()
                    dataBuf,pos_X,pos_Y,pos_Z,Euler=dd.solveData(data)
                    if(SendTargetPos == 1):           
                        posZ=0.0
                        dataBuf[43] = 0x20
                        posX_buf = struct.pack("f",targetx) 
                        dataBuf[44] = posX_buf[0]
                        dataBuf[45] = posX_buf[1]
                        dataBuf[46] = posX_buf[2]
                        dataBuf[47] = posX_buf[3] 
                        posY_buf = struct.pack("f",targety)
                        dataBuf[48] = posY_buf[0]
                        dataBuf[49] = posY_buf[1]
                        dataBuf[50] = posY_buf[2]
                        dataBuf[51] = posY_buf[3]
                        posZ_buf = struct.pack("f",posZ) 
                        dataBuf[52] = posZ_buf[0]
                        dataBuf[53] = posZ_buf[1]
                        dataBuf[54] = posZ_buf[2]
                        dataBuf[55] = posZ_buf[3]

                        dataBuf[56] = LaserArray              
                        Laser_Dis = struct.pack("f",LaserDistance)
                        dataBuf[57] = Laser_Dis[0]
                        dataBuf[58] = Laser_Dis[1]
                        dataBuf[59] = Laser_Dis[2]             
                        dataBuf[60] = Laser_Dis[3]                  
                        dataBuf[61] = FlightMode
                   
                    if CopterLanding==1:
                        dataBuf[62] = 0xA5
                    else:
                        dataBuf[62] = 0x00
                    
                    for i in range(0,62):                     
                        CheckSum =  CheckSum + dataBuf[i]
                
                    dataBuf[63] = 0xAA
                    dataBuf[64] = CheckSum & 0x00ff
                    
                    print("\rrpy_rad[0]:{:.2f},rpy_rad[1]:{:.2f},rpy_rad[2]:{:.2f} ,X:{:.2f},Y:{:.2f},Z:{:.2f} ".format(Euler[0]*57.3,Euler[1]*57.3,Euler[2]*57.3,pos_Y,-pos_X,pos_Z),end="")
                    port.write(dataBuf)
                    CheckSum = 0
            else:
                dataBuf[0] = 0x55
                dataBuf[1] = 0xAA
                dataBuf[2] = 0xFF
                dataBuf[63] = 0xAA
                dataBuf[64] = 0x00
                port.write(dataBuf)
                time.sleep(0.1)
    finally:
        
        pipe.stop()
        dataBuf[62] = 0xA5
        port.write(dataBuf)
        print("some erro")


0 b'Start265\n' b'Start265\n'
rpy_rad[0]:-4.98,rpy_rad[1]:-0.38,rpy_rad[2]:-0.13 ,X:-0.01,Y:-0.07,Z:1.34  b'Departures\n' b'Departures\n'
Get!
rpy_rad[0]:-5.06,rpy_rad[1]:-0.34,rpy_rad[2]:0.29 ,X:-0.00,Y:-0.08,Z:1.37  

Image(value=b'')

rpy_rad[0]:-5.05,rpy_rad[1]:-0.34,rpy_rad[2]:0.29 ,X:-0.00,Y:-0.08,Z:1.37 route node 2: x : -1.5 , y : 0.0  
rpy_rad[0]:-4.65,rpy_rad[1]:-0.57,rpy_rad[2]:1.46 ,X:-1.41,Y:-0.03,Z:1.38 route node 3: x : -1.5 , y : 1.4  
rpy_rad[0]:-4.68,rpy_rad[1]:-0.83,rpy_rad[2]:2.23 ,X:-1.54,Y:1.31,Z:1.37 route node 4: x : -2.5 , y : 1.4  
rpy_rad[0]:-5.07,rpy_rad[1]:-0.05,rpy_rad[2]:0.34 ,X:-2.43,Y:1.37,Z:1.37 
识别形状triangle按照比赛要求火情最小
route node 5: x : -1.5 , y : 1.4  
rpy_rad[0]:-3.88,rpy_rad[1]:-0.11,rpy_rad[2]:0.70 ,X:-1.57,Y:1.47,Z:1.37 route node 6: x : -1.5 , y : 2.4  
rpy_rad[0]:-4.02,rpy_rad[1]:-1.10,rpy_rad[2]:3.13 ,X:-1.55,Y:2.31,Z:1.33 route node 7: x : -0.5 , y : 2.4  
rpy_rad[0]:-4.05,rpy_rad[1]:-0.30,rpy_rad[2]:1.92 ,X:-0.55,Y:2.33,Z:1.28 
识别形状circle按照比赛要求火情次之
route node 8: x : -0.5 , y : 3.4  
rpy_rad[0]:-3.97,rpy_rad[1]:-0.09,rpy_rad[2]:3.74 ,X:-0.43,Y:3.33,Z:1.32 route node 9: x : -2.5 , y : 3.4  
rpy_rad[0]:-3.82,rpy_rad[1]:0.61,rpy_rad[2]:5.97 ,X:-2.39,Y:3.37,Z:1.29  
识别形状square按照比赛

RuntimeError: Frame didn't arrive within 5000


circle
